In [1]:
# Check for GPU
# !nvidia-smi -L
!nvidia-smi 

Thu Jan 13 14:19:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Get helper functions



In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-01-13 14:19:49--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-01-13 14:19:49 (103 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import unzip_data,create_tensorboard_callback,plot_loss_curves,compare_historys

# Get Dataset

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2022-01-13 14:19:51--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.193.128, 172.217.204.128, 172.217.203.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.193.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-01-13 14:19:51 (145 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
# Unzip Data

unzip_data( 'nlp_getting_started.zip')

# Visualizing the Text Data

In [6]:
import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [7]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
train_df.text[1]

'Forest fire near La Ronge Sask. Canada'

In [9]:
train_df_shuffled = train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [10]:
# Getting femiliar with Test Data

test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [11]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [12]:
# How many samples?

len(train_df_shuffled), len(test_df)

(7613, 3263)

In [13]:
# Let's visualize some random training samples

import random
random_index = random.randint(0, len(train_df_shuffled)-5)
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
  _,text,target = row
  print(f'Target: {target}', "(Real Disaster)" if target == 1 else "(Not a Real Disaster")
  print(f'Text:\n{text}\n')
  print(f'----\n')

Target: 1 (Real Disaster)
Text:
'Food crematoria' in Russia provoke outrage amid crisis famine memories
http://t.co/FelR5a1hBP

----

Target: 1 (Real Disaster)
Text:
The Latest: More homes razed by Northern California wildfire - http://t.co/BsGR67dyWY http://t.co/nDGgr6Xyqd

----

Target: 0 (Not a Real Disaster
Text:
--thus making @FemaleGilgamesh's assault useless.
The spears collided with the dark force however did not penetrate.

Due to the dark --

----

Target: 0 (Not a Real Disaster
Text:
@blazerfan not everyone can see ignoranceshe is Latinoand that is All she can ever benothing morebut an attack dog 4 a hate group GOP

----

Target: 0 (Not a Real Disaster
Text:
@BehindAShield @Wars_Goddess Sweet Lord.  (I collapse as my knees buckle)

----



# split data into training and validation dataset

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_sentences,val_sentences,train_labels,val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                         train_df_shuffled['target'].to_numpy(),
                                                                         test_size=0.1, random_state=42)

In [16]:
len(train_sentences),len(val_sentences),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

# Converting text into numbers

In [17]:
# text Tokenization
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [18]:
text_vectorizer = TextVectorization(max_tokens=10000,
                                    standardize='lower_and_strip_punctuation',
                                    split= "whitespace",
                                    ngrams = None,
                                    output_mode = 'int',
                                    output_sequence_length = 15,
                                    pad_to_max_tokens = True)

In [19]:
text_vectorizer.adapt(train_sentences)

In [20]:
random_sentence = random.choice(train_sentences)
print(f"Original Sentence: \n {random_sentence}\
      \n\nVectorized Version")
text_vectorizer([random_sentence])

Original Sentence: 
 Just down drown me k I can't swim https://t.co/sJoEing76t      

Vectorized Version


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  29,  134,  486,   31,    1,    8,   98, 2272,    1,    0,    0,
           0,    0,    0,    0]])>

In [21]:
# GEt the Unique words from vocabulaty
words_in_vocab = text_vectorizer.get_vocabulary()
top_10 = words_in_vocab[:10]
least_10 = words_in_vocab[-10:]
print(f"No of words in vocab: {len(words_in_vocab)}")
print(f"Top 10 repeating words: {top_10}")
print(f"Least 10 repeating words: {least_10}")

No of words in vocab: 10000
Top 10 repeating words: ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']
Least 10 repeating words: ['painthey', 'painful', 'paine', 'paging', 'pageshi', 'pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


# Creating and using ebedding layer

In [22]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = 10000,
                             output_dim = 128,
                             input_length = 15)

In [23]:
embedding

In [24]:
# GEt random sentence from the training set

random_sentence = random.choice(train_sentences)

print(f'Original sentence: \n {random_sentence}\
      \n\n Embedded Sentence:\n')

embedding(text_vectorizer([random_sentence]))

Original sentence: 
 Enugu Government to demolish illegal structures at International Conference Centre http://t.co/7K5SHaiqIw      

 Embedded Sentence:



<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.02634182,  0.02373132,  0.03788872, ..., -0.04460553,
         -0.02857485, -0.04360295],
        [ 0.04992932,  0.0224409 ,  0.03780236, ...,  0.0356988 ,
          0.00044657,  0.04913366],
        [ 0.01730398,  0.02865735, -0.01327353, ..., -0.00553634,
         -0.0359241 , -0.01727753],
        ...,
        [ 0.00450033, -0.00124891, -0.03897219, ...,  0.01418198,
          0.04606939, -0.03713702],
        [ 0.00450033, -0.00124891, -0.03897219, ...,  0.01418198,
          0.04606939, -0.03713702],
        [ 0.00450033, -0.00124891, -0.03897219, ...,  0.01418198,
          0.04606939, -0.03713702]]], dtype=float32)>

# Model Building
- Base line naive bayes
- feedforward 
- LSTM
- GRU
- Bidirectional LSTM
- 1D Convolutional 
- Tensorflow hub - pretrained feature extractor - transfer learing
- 

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create a Tokenization and modeling pipeline
model_0 = Pipeline([
                    ('tfidf', TfidfVectorizer()),
                    ('clf', MultinomialNB())
])

# Fit the data into the pipeline

model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [26]:
# Evluate our trained model
baseline_score = model_0.score(val_sentences,val_labels)
print(f'Our baseline model achieves an accuraty of: {baseline_score * 100:.2f}%')

Our baseline model achieves an accuraty of: 79.27%


In [27]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [28]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, precision_recall_fscore_support
def metrics(y_true,y_pred):
  accuracy = accuracy_score(y_true,y_pred)
  # precision = precision_score(y_true,y_pred)
  # recall = recall_score(y_true,y_pred)
  Preci, recal, f1, suppo = precision_recall_fscore_support(y_true,y_pred,average='weighted')
  # f1score = f1_score(y_true,y_pred)
  # model_metrics = {'ModelAccuracy' : accuracy, 'ModelPrecision' : precision, 'ModelRecall' : recall, 'ModelF1_Score' : f1score}
  prfs = { 'accuracy':accuracy, 'precision': Preci, 'recall':recal, 'f1':f1}
  return prfs #model_metrics,

In [29]:
metrics(baseline_preds,val_labels)

{'accuracy': 0.7926509186351706,
 'f1': 0.7990828614653861,
 'precision': 0.8336022277575122,
 'recall': 0.7926509186351706}

In [30]:
baseline_results = metrics(val_labels,baseline_preds)
baseline_results

{'accuracy': 0.7926509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

# Simple ANN model on text data

In [31]:
from helper_functions import create_tensorboard_callback

SAVE_DIR = 'model_logs'

In [32]:
# build a model with functional API
from tensorflow.keras import layers

inputs = layers.Input(shape = (1), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
output = layers.Dense(1, activation='sigmoid')(x)
model_1 = tf.keras.Model(inputs,output, name = 'model_1_dense')

In [33]:
model_1.compile(optimizer=tf.keras.optimizers.Adam(),
                loss = 'binary_crossentropy',
                metrics = ['accuracy'])

In [34]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [35]:
model_1.fit(x = train_sentences,
            y = train_labels,
            epochs = 5,
            workers = -1,
            validation_data = (val_sentences,val_labels),
            callbacks = [create_tensorboard_callback(dir_name= SAVE_DIR,
                                                     experiment_name = 'simple_dense_model')])

Saving TensorBoard log files to: model_logs/simple_dense_model/20220113-141955
Epoch 1/5
215/215 [==============================] - 3s 5ms/step - loss: 0.6139 - accuracy: 0.6866 - val_loss: 0.5364 - val_accuracy: 0.7480
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.4422 - accuracy: 0.8177 - val_loss: 0.4735 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3477 - accuracy: 0.8628 - val_loss: 0.4567 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2849 - accuracy: 0.8901 - val_loss: 0.4655 - val_accuracy: 0.7927
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2389 - accuracy: 0.9115 - val_loss: 0.4771 - val_accuracy: 0.7795


In [36]:
model_1_pred = model_1.predict(val_sentences)

In [37]:
model_1_pred = tf.squeeze(tf.round(model_1_pred))

In [38]:
model_1.evaluate(x = val_sentences, y = val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4771 - accuracy: 0.7795


[0.47710707783699036, 0.7795275449752808]

In [39]:
embedding.weights

[<tf.Variable 'embedding/embeddings:0' shape=(10000, 128) dtype=float32, numpy=
 array([[ 2.1115635e-02,  2.0398531e-02, -2.1076923e-02, ...,
         -5.6329975e-03,  6.9153100e-02, -5.8080688e-02],
        [-4.5399036e-02, -9.7964844e-03,  1.9743474e-04, ...,
         -5.0190769e-02, -1.5451603e-02, -9.3217893e-04],
        [-2.1233995e-02,  2.8064664e-02,  7.0239916e-03, ...,
         -2.8121516e-02,  1.0567012e-02, -6.7916112e-03],
        ...,
        [ 3.3237521e-02, -5.8568493e-03, -2.1959687e-02, ...,
         -2.0762110e-02,  1.2003519e-02, -4.6805382e-02],
        [ 9.3416814e-03,  3.4597151e-02, -6.3368352e-05, ...,
         -5.6785345e-02,  5.4657371e-03, -8.4668905e-02],
        [ 4.0366869e-02,  5.1979441e-02,  7.4103862e-02, ...,
         -6.6808186e-02,  5.8295533e-02, -7.0267774e-02]], dtype=float32)>]

In [40]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [41]:
embed_weights = model_1.get_layer('embedding').get_weights()[0]
embed_weights, type(embed_weights), embed_weights.shape

(array([[ 2.1115635e-02,  2.0398531e-02, -2.1076923e-02, ...,
         -5.6329975e-03,  6.9153100e-02, -5.8080688e-02],
        [-4.5399036e-02, -9.7964844e-03,  1.9743474e-04, ...,
         -5.0190769e-02, -1.5451603e-02, -9.3217893e-04],
        [-2.1233995e-02,  2.8064664e-02,  7.0239916e-03, ...,
         -2.8121516e-02,  1.0567012e-02, -6.7916112e-03],
        ...,
        [ 3.3237521e-02, -5.8568493e-03, -2.1959687e-02, ...,
         -2.0762110e-02,  1.2003519e-02, -4.6805382e-02],
        [ 9.3416814e-03,  3.4597151e-02, -6.3368352e-05, ...,
         -5.6785345e-02,  5.4657371e-03, -8.4668905e-02],
        [ 4.0366869e-02,  5.1979441e-02,  7.4103862e-02, ...,
         -6.6808186e-02,  5.8295533e-02, -7.0267774e-02]], dtype=float32),
 numpy.ndarray,
 (10000, 128))

In [42]:
model_1_results = metrics(val_labels,model_1_pred)
model_1_results

{'accuracy': 0.7795275590551181,
 'f1': 0.7771404562571971,
 'precision': 0.7822644211580037,
 'recall': 0.7795275590551181}

In [43]:
def compare_baseline_to_new_results(baseline_results,model_1_results):
   for key, value in baseline_results.items():
     print(f'Baseline {key:>10}: {value:.2f}   New {key:>10}: {model_1_results[key]:.2f},   Difference {model_1_results[key]-value:.2f}')

In [44]:
compare_baseline_to_new_results(baseline_results,model_1_results)

Baseline   accuracy: 0.79   New   accuracy: 0.78,   Difference -0.01
Baseline  precision: 0.81   New  precision: 0.78,   Difference -0.03
Baseline     recall: 0.79   New     recall: 0.78,   Difference -0.01
Baseline         f1: 0.79   New         f1: 0.78,   Difference -0.01


In [45]:
words_in_vocab[:10]

['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']

In [46]:
# Get the weight matrix of the embedding layer
embed_weights = model_1.get_layer('embedding').get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [47]:
# import io

# # Create output writers
# out_v = io.open("embedding_vectors.tsv", "w", encoding="utf-8")
# out_m = io.open("embedding_metadata.tsv", "w", encoding="utf-8")

# # Write embedding vectors and words to file
# for num, word in enumerate(words_in_vocab):
#   if num == 0: 
#      continue # skip padding token
#   vec = embed_weights[num]
#   out_m.write(word + "\n") # write words to file
#   out_v.write("\t".join([str(x) for x in vec]) + "\n") # write corresponding word vector to file
# out_v.close()
# out_m.close()

In [48]:
# Download files locally to upload to Embedding Projector
# try:
#   from google.colab import files
# except ImportError:
#   pass
# else:
#   files.download("embedding_vectors.tsv")
#   files.download("embedding_metadata.tsv")

In [49]:
tf.random.set_seed(42)
max_vocab_length = 10000
max_length = 16
model_2_embedding = layers.Embedding(input_dim = max_vocab_length,
                                     output_dim = 128,
                                     embeddings_initializer = 'uniform',
                                     input_length = max_length,
                                     name = 'embedding_2')

# Create LSTM model
inputs = layers.Input(shape=(1),dtype='string',name = 'inputs_2') #,batch_size=10
x = text_vectorizer(inputs)
x = model_2_embedding(x)
print(x.shape)
x = layers.LSTM(128,return_sequences=True)(x)
print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
x = layers.Dense(128,activation='relu')(x)
print(x.shape)
outputs = layers.Dense(1,activation='sigmoid')(x)
print(outputs.shape)
model_2 = tf.keras.Model(inputs,outputs, name = 'model_2_LSTM')

(None, 15, 128)
(None, 15, 128)
(None, 64)
(None, 128)
(None, 1)


In [50]:
model_2.compile(loss='binary_crossentropy',
                optimizer = tf.optimizers.Adam(),
                metrics = ['accuracy'])

In [51]:
# Before we fit the model let us check the summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs_2 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 15, 128)           131584    
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                      

In [52]:
model_2.fit(x = train_sentences,
            y = train_labels,
            epochs = 5,
            validation_data = (val_sentences,val_labels),
            callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                     'LSTM')])

Saving TensorBoard log files to: model_logs/LSTM/20220113-142003
Epoch 1/5
215/215 [==============================] - 7s 12ms/step - loss: 0.5111 - accuracy: 0.7503 - val_loss: 0.4571 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.3276 - accuracy: 0.8702 - val_loss: 0.5174 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2275 - accuracy: 0.9118 - val_loss: 0.5520 - val_accuracy: 0.7638
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1660 - accuracy: 0.9437 - val_loss: 0.6053 - val_accuracy: 0.7664
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1167 - accuracy: 0.9580 - val_loss: 0.7879 - val_accuracy: 0.7612


In [53]:
model_2_pred_probs = model_2.predict(val_sentences)
print(model_2_pred_probs[:10])
model_2_pred = tf.squeeze(tf.round(model_2_pred_probs))
model_2_pred[:10]

[[5.76038249e-02]
 [5.47399640e-01]
 [9.98732388e-01]
 [1.50234893e-01]
 [8.32074380e-04]
 [9.95378852e-01]
 [4.29290861e-01]
 [9.95184124e-01]
 [9.98547375e-01]
 [1.07939236e-01]]


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 0.], dtype=float32)>

In [54]:
model_2_results = metrics(val_labels,model_2_pred)
model_2_results

{'accuracy': 0.7611548556430446,
 'f1': 0.7577956328280006,
 'precision': 0.7649393219101418,
 'recall': 0.7611548556430446}

In [55]:
compare_baseline_to_new_results(baseline_results,model_2_results)

Baseline   accuracy: 0.79   New   accuracy: 0.76,   Difference -0.03
Baseline  precision: 0.81   New  precision: 0.76,   Difference -0.05
Baseline     recall: 0.79   New     recall: 0.76,   Difference -0.03
Baseline         f1: 0.79   New         f1: 0.76,   Difference -0.03


# GRU Model

In [56]:
tf.random.set_seed(42)
model_3_embedding = layers.Embedding(input_dim = max_vocab_length,
                                     output_dim = 128,
                                     embeddings_initializer = 'uniform',
                                     input_length = max_length,
                                     name = 'embedding_3')

# Build GRU model
inputs = layers.Input(shape = (1,),
                     name = 'input_3',
                     dtype = 'string')
x = text_vectorizer(inputs)
x = model_3_embedding(x)
print(x.shape)
# x = layers.GRU(64, return_sequences=True)(x)
x = layers.GRU(64)(x)
print(x.shape)
output = layers.Dense(1,activation = 'sigmoid')(x)
print(output.shape)
model_3 = tf.keras.Model(inputs,output,name='model_GRU')

(None, 15, 128)
(None, 64)
(None, 1)


In [57]:
model_3.compile(optimizer=tf.keras.optimizers.Adam(),
                loss = 'binary_crossentropy',
                metrics = ['accuracy'])

In [58]:
model_3.summary()

Model: "model_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_______________________________________________

In [59]:
model_3.fit(x = train_sentences,
            y = train_labels,
            epochs = 5,
            validation_data = (val_sentences,val_labels),
            callbacks = [create_tensorboard_callback(SAVE_DIR,'GRU')],
            batch_size = 15)

Saving TensorBoard log files to: model_logs/GRU/20220113-142019
Epoch 1/5
457/457 [==============================] - 5s 6ms/step - loss: 0.5064 - accuracy: 0.7470 - val_loss: 0.4489 - val_accuracy: 0.7887
Epoch 2/5
457/457 [==============================] - 3s 6ms/step - loss: 0.3175 - accuracy: 0.8689 - val_loss: 0.5255 - val_accuracy: 0.7717
Epoch 3/5
457/457 [==============================] - 3s 6ms/step - loss: 0.2152 - accuracy: 0.9188 - val_loss: 0.5743 - val_accuracy: 0.7730
Epoch 4/5
457/457 [==============================] - 3s 6ms/step - loss: 0.1502 - accuracy: 0.9482 - val_loss: 0.6356 - val_accuracy: 0.7703
Epoch 5/5
457/457 [==============================] - 3s 6ms/step - loss: 0.1069 - accuracy: 0.9613 - val_loss: 0.6936 - val_accuracy: 0.7756


In [60]:
model_3_pred_proba = model_3.predict(val_sentences)
print(f'Pridicted probabilities\n {model_3_pred_proba[:5]} \n\n Pridicated class')
model_3_pred = tf.squeeze(tf.round(model_3_pred_proba))
model_3_pred[:10]

Pridicted probabilities
 [[0.08677846]
 [0.90265805]
 [0.9969765 ]
 [0.14623697]
 [0.00297258]] 

 Pridicated class


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [61]:
# calculate metrics

model_3_results = metrics(y_true = val_labels,
                          y_pred = model_3_pred)
model_3_results

{'accuracy': 0.7755905511811023,
 'f1': 0.7738609431347062,
 'precision': 0.7767334207253159,
 'recall': 0.7755905511811023}

# Conv 1D model

In [62]:
# Build a CNN 1D Model
tf.random.set_seed(42)
model_4_embedding = layers.Embedding(input_dim = max_vocab_length,
                                     output_dim = 128,
                                     embeddings_initializer = 'uniform',
                                     input_length = max_length,
                                     name = 'embedding_4')

inputs = layers.Input(shape=(1), name = 'inputlayer', dtype='string')
x = text_vectorizer(inputs)
x = model_4_embedding(x)
print(x.shape)
conv_output = layers.Conv1D(64, 4, padding = 'same',activation='relu')
x = conv_output(x)
print(x.shape)
maxpool_output = layers.GlobalMaxPool1D()
x = maxpool_output(x)
print(x.shape)
outputs = layers.Dense(1,activation= 'sigmoid')(x)
model_4 = tf.keras.Model(inputs, outputs, name = 'CNN1D')

model_4.compile(optimizer = tf.keras.optimizers.Adam(),
                loss = 'binary_crossentropy',
                metrics = ['accuracy'])

model_4.summary()

(None, 15, 128)
(None, 15, 64)
(None, 64)
Model: "CNN1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputlayer (InputLayer)     [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_4 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 conv1d (Conv1D)             (None, 15, 64)            32832     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense_4 (Dense)   

In [63]:
model_4.fit(x = train_sentences, y = train_labels,
            epochs = 1, validation_data = (val_sentences,val_labels),
            workers = -1,
            callbacks = [create_tensorboard_callback(SAVE_DIR,'Conv1D')])

Saving TensorBoard log files to: model_logs/Conv1D/20220113-142034
215/215 [==============================] - 8s 6ms/step - loss: 0.5565 - accuracy: 0.7209 - val_loss: 0.4648 - val_accuracy: 0.7769


In [64]:
model_4_pred_proba = model_4.predict(val_sentences)
print(model_4_pred_proba[:5])
model_4_pred = tf.squeeze(tf.round(model_4_pred_proba))
print(model_4_pred[:5])

[[0.3841609 ]
 [0.7147779 ]
 [0.9489205 ]
 [0.25828883]
 [0.23438701]]
tf.Tensor([0. 1. 1. 0. 0.], shape=(5,), dtype=float32)


In [65]:
model_4_results = metrics(y_true=val_labels,y_pred=model_4_pred)
model_4_results

{'accuracy': 0.7769028871391076,
 'f1': 0.774064384984916,
 'precision': 0.7805885493126917,
 'recall': 0.7769028871391076}

# Transfer learning - Universal Sentence Encoder (USE)

In [66]:
import tensorflow_hub as hub
# model_5_embedding = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')

# # create a model
# inputs = layers.Input(shape = (1), batch_size=None,name=None,dtype=tf.string)
# # x = text_vectorizer(inputs)
# x = model_5_embedding(inputs)
# print(x.shape)
# x = layers.Dense(1, activation = 'sigmoid')
# print(x.shape)

In [67]:
model_5_embedding = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4',
                                   trainable = False,
                                   dtype = 'string', input_shape = [],
                                   name = 'USE_sentence_encoder')

In [68]:
# create a model

model_5 = tf.keras.Sequential([
                               model_5_embedding,
                               layers.Dense(512, activation = 'relu'),
                              #  layers.GlobalAveragePooling1D(),
                               layers.Dense(1,activation= 'sigmoid')
])

model_5.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

model_5.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE_sentence_encoder (Keras  (None, 512)              256797824 
 Layer)                                                          
                                                                 
 dense_5 (Dense)             (None, 512)               262656    
                                                                 
 dense_6 (Dense)             (None, 1)                 513       
                                                                 
Total params: 257,060,993
Trainable params: 263,169
Non-trainable params: 256,797,824
_________________________________________________________________


In [69]:
model_5.fit(x = train_sentences,
            y = train_labels,
            epochs = 10,
            validation_data = (val_sentences,val_labels),
            callbacks = [create_tensorboard_callback(SAVE_DIR,'Sentence_encoderUSE')])

Saving TensorBoard log files to: model_logs/Sentence_encoderUSE/20220113-142100
Epoch 1/10
215/215 [==============================] - 5s 13ms/step - loss: 0.4625 - accuracy: 0.7968 - val_loss: 0.4389 - val_accuracy: 0.8058
Epoch 2/10
215/215 [==============================] - 2s 11ms/step - loss: 0.3990 - accuracy: 0.8219 - val_loss: 0.4241 - val_accuracy: 0.8228
Epoch 3/10
215/215 [==============================] - 2s 11ms/step - loss: 0.3764 - accuracy: 0.8354 - val_loss: 0.4215 - val_accuracy: 0.8255
Epoch 4/10
215/215 [==============================] - 2s 11ms/step - loss: 0.3561 - accuracy: 0.8459 - val_loss: 0.4206 - val_accuracy: 0.8215
Epoch 5/10
215/215 [==============================] - 2s 11ms/step - loss: 0.3312 - accuracy: 0.8562 - val_loss: 0.4221 - val_accuracy: 0.8202
Epoch 6/10
215/215 [==============================] - 2s 12ms/step - loss: 0.3045 - accuracy: 0.8756 - val_loss: 0.4225 - val_accuracy: 0.8176
Epoch 7/10
215/215 [==============================] - 2s 11ms/

In [70]:
model_5_pred_proba = model_5.predict(val_sentences)
print(model_5_pred_proba[:5])
model_5_pred = tf.squeeze(tf.round(model_5_pred_proba))
model_5_pred[:10]

[[0.04553965]
 [0.71192884]
 [0.9995608 ]
 [0.3485214 ]
 [0.36715168]]


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [71]:
model_5_results = metrics(y_true=val_labels,y_pred=model_5_pred)
model_5_results

{'accuracy': 0.8241469816272966,
 'f1': 0.8211689033722674,
 'precision': 0.8329324306546825,
 'recall': 0.8241469816272966}

In [72]:
# train_df_10_percent = train_df_shuffled[['text','target']].sample(frac = 0.1, random_state=42)
# len(train_df_10_percent)

train_sent_90_percent, train_sent_10_percent, train_label_90_percent, train_label_10_percent = train_test_split(train_sentences,
                                                                                                                train_labels,
                                                                                                                test_size = .1,random_state = 42)
train_sent_10_percent.shape,train_label_10_percent.shape

((686,), (686,))

In [73]:
temp = pd.Series(train_label_10_percent).value_counts()
temp[0],temp[1]

(415, 271)

In [74]:
temp[1]/(temp[0]+temp[1])

0.39504373177842567

# Build a transfer learning model with 10% data

In [75]:
model_6_embedding = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4',
                                   input_shape = [],
                                   dtype = 'string',
                                   trainable = False,
                                   name = 'USE_embedding_10percent')

In [76]:
model_6 = tf.keras.Sequential([
                                model_6_embedding,
                                layers.Dense(1024,activation='relu'),
                               layers.Dense(256, activation='relu'),
                                layers.Dense(1,activation='sigmoid')
], name = 'model6_10percent_USE')

model_6.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
                metrics = ['accuracy'])

model_6.summary()

Model: "model6_10percent_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE_embedding_10percent (Ke  (None, 512)              256797824 
 rasLayer)                                                       
                                                                 
 dense_7 (Dense)             (None, 1024)              525312    
                                                                 
 dense_8 (Dense)             (None, 256)               262400    
                                                                 
 dense_9 (Dense)             (None, 1)                 257       
                                                                 
Total params: 257,585,793
Trainable params: 787,969
Non-trainable params: 256,797,824
_________________________________________________________________


In [77]:
model_6.fit(x = train_sent_10_percent,
            y = train_label_10_percent,
            epochs = 10,
            validation_data = (val_sentences,val_labels),
            callbacks = [create_tensorboard_callback(SAVE_DIR,'USE10percentdata')])

Saving TensorBoard log files to: model_logs/USE10percentdata/20220113-142131
Epoch 1/10
22/22 [==============================] - 3s 44ms/step - loss: 0.5171 - accuracy: 0.7464 - val_loss: 0.4750 - val_accuracy: 0.7795
Epoch 2/10
22/22 [==============================] - 1s 25ms/step - loss: 0.3106 - accuracy: 0.8688 - val_loss: 0.5374 - val_accuracy: 0.7927
Epoch 3/10
22/22 [==============================] - 1s 25ms/step - loss: 0.1275 - accuracy: 0.9621 - val_loss: 0.8537 - val_accuracy: 0.7546
Epoch 4/10
22/22 [==============================] - 0s 21ms/step - loss: 0.0715 - accuracy: 0.9840 - val_loss: 1.0640 - val_accuracy: 0.7717
Epoch 5/10
22/22 [==============================] - 1s 25ms/step - loss: 0.0624 - accuracy: 0.9767 - val_loss: 1.3478 - val_accuracy: 0.7480
Epoch 6/10
22/22 [==============================] - 0s 21ms/step - loss: 0.0820 - accuracy: 0.9708 - val_loss: 1.3942 - val_accuracy: 0.7454
Epoch 7/10
22/22 [==============================] - 1s 26ms/step - loss: 0.09

In [78]:
model_6_pred_proba = model_6.predict(val_sentences)
print(model_6_pred_proba[:5])
model_6_pred = tf.squeeze(tf.round(model_6_pred_proba))
model_6_pred[:10]

[[6.8213695e-01]
 [1.0000000e+00]
 [1.0000000e+00]
 [4.0208219e-04]
 [1.7208018e-03]]


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [79]:
model_6_results = metrics(val_labels,model_6_pred)
model_6_results

{'accuracy': 0.7664041994750657,
 'f1': 0.7662288382234984,
 'precision': 0.7661489682210776,
 'recall': 0.7664041994750657}

In [80]:
import tensorflow as tf

In [81]:
x = tf.constant(4,shape=(5))
x

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([4, 4, 4, 4, 4], dtype=int32)>

In [82]:
import numpy as np
np.array([[1,2,3,4],[4,5,6,7]]).shape

(2, 4)

In [83]:
np.array([4,6,8,10]).T.shape

(4,)

In [84]:
print(np.array([4,5,6,7,8]).T)
print(np.array([[[1,2],[4,5]],[[1,2],[4,5,6]]]).T)

[4 5 6 7 8]
[[list([1, 2]) list([1, 2])]
 [list([4, 5]) list([4, 5, 6])]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [85]:
model = tf.keras.Sequential()
# model.add(tf.keras.Input(shape=(4,)))
# model.summary()
# print("-----")
model.add(tf.keras.layers.Dense(2, activation="relu", input_shape=(4,1)))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 4, 2)              4         
                                                                 
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________
